In [ ]:


import pandas as pd
import io

# --- Simulate loading your source types CSV data ---
# csv_data = """
# "id","parent_id1","parent_id2","parent_id3","name","label"
# 116,97,75,115,"PŘ – alkohol ostatní","Alkohol ostatní"
# 97,0,0,0,"Přestupky","Přestupky"
# 75,97,0,0,"PŘ – alkohol a toxi","Na úseku ochrany před alkoholismem toxikomanií"
# 115,97,75,0,"PŘ – alkohol ostatní","Alkohol ostatní"
# """
# df_source = pd.read_csv(io.StringIO(csv_data))
df_source = pd.read_csv("C://Users//jirip//Documents//Developer//python//kriminalita//types.csv", sep=",", encoding="utf-8")


# Replace 0 parent IDs with NaN
df_source['parent_id1'] = df_source['parent_id1'].replace(0, pd.NA)
df_source['parent_id2'] = df_source['parent_id2'].replace(0, pd.NA)
df_source['parent_id3'] = df_source['parent_id3'].replace(0, pd.NA)

# --- Prepare the dimension data ---
df_dim = df_source[['id', 'name', 'label', 'parent_id1', 'parent_id2', 'parent_id3']].copy()
df_dim.rename(columns={
    'id': 'Level4_ID', 'name': 'Level4_Name', 'label': 'Level4_Label'
}, inplace=True)

df_lookup = df_source[['id', 'name', 'label']].copy()

# Merge to get Level 3, 2, 1 attributes (as before)
df_dim = pd.merge(df_dim, df_lookup.rename(columns={'id': 'Level3_ID', 'name': 'Level3_Name', 'label': 'Level3_Label'}), left_on='parent_id3', right_on='Level3_ID', how='left')
df_dim = pd.merge(df_dim, df_lookup.rename(columns={'id': 'Level2_ID', 'name': 'Level2_Name', 'label': 'Level2_Label'}), left_on='parent_id2', right_on='Level2_ID', how='left')
df_dim = pd.merge(df_dim, df_lookup.rename(columns={'id': 'Level1_ID', 'name': 'Level1_Name', 'label': 'Level1_Label'}), left_on='parent_id1', right_on='Level1_ID', how='left')

# Add SourceTypeID
df_dim['SourceTypeID'] = df_dim['Level4_ID']

# --- *** ADD TypeSK GENERATION HERE *** ---
# Option A: Using reset_index (starts SK from 0)
# df_dim = df_dim.reset_index()
# df_dim.rename(columns={'index': 'TypeSK'}, inplace=True)

# Option B: Creating a sequence (starts SK from 1 - often preferred for DW keys)
df_dim.insert(0, 'TypeSK', range(1, len(df_dim) + 1))


# --- Finalize the dimension DataFrame ---
# Calculate optional fields (Depth, Path)
def calculate_depth(row):
    # Note: Adjusted depth calculation slightly to be more robust
    depth = 0
    if pd.notna(row['Level1_ID']) or pd.notna(row['Level1_Name']): depth = 1 # If L1 exists, depth is at least 1
    if pd.notna(row['Level2_ID']) or pd.notna(row['Level2_Name']): depth = 2 # If L2 exists, depth is at least 2
    if pd.notna(row['Level3_ID']) or pd.notna(row['Level3_Name']): depth = 3 # If L3 exists, depth is at least 3
    if pd.notna(row['Level4_ID']) or pd.notna(row['Level4_Name']): depth = 4 # If L4 exists, depth is at least 4
    # Handle root nodes correctly (where L1 is populated but others might be NA)
    if depth == 0 and (pd.notna(row['Level4_ID']) or pd.notna(row['Level4_Name'])):
        depth = 1 # If only L4 details exist somehow (e.g. root node entered weirdly) treat as depth 1
    return depth if depth > 0 else 1 # Ensure minimum depth 1 if valid L4 exists


def calculate_path(row, field='Name'):
    parts = []
    if pd.notna(row[f'Level1_{field}']): parts.append(str(row[f'Level1_{field}']))
    if pd.notna(row[f'Level2_{field}']): parts.append(str(row[f'Level2_{field}']))
    if pd.notna(row[f'Level3_{field}']): parts.append(str(row[f'Level3_{field}']))
    if pd.notna(row[f'Level4_{field}']): parts.append(str(row[f'Level4_{field}']))
    return ' > '.join(parts) if parts else None

df_dim['HierarchyDepth'] = df_dim.apply(calculate_depth, axis=1)
df_dim['FullHierarchyPathName'] = df_dim.apply(lambda row: calculate_path(row, 'Name'), axis=1)
df_dim['FullHierarchyPathLabel'] = df_dim.apply(lambda row: calculate_path(row, 'Label'), axis=1)


# Select and order final columns for DimType table
final_dim_columns = [
    'TypeSK', 'SourceTypeID',
    'Level1_ID', 'Level1_Name', 'Level1_Label',
    'Level2_ID', 'Level2_Name', 'Level2_Label',
    'Level3_ID', 'Level3_Name', 'Level3_Label',
    'Level4_ID', 'Level4_Name', 'Level4_Label',
    'FullHierarchyPathName', 'FullHierarchyPathLabel', 'HierarchyDepth'
]
df_dim_type = df_dim[final_dim_columns].copy()

# Convert Pandas <NA> or NaN to None for database compatibility
df_dim_type = df_dim_type.astype(object).where(pd.notnull(df_dim_type), None)


print("--- Final DimType DataFrame (with TypeSK) ---")
print(df_dim_type.to_string())


In [ ]:
import pandas as pd
import io

# --- Assume DimType DataFrame 'df_dim_type' is already created ---
# Example df_dim_type (needs SourceTypeID, TypeSK, HierarchyDepth)
# dim_data = {
#     'TypeSK': [101, 102, 103, 104], # Example Surrogate Keys
#     'SourceTypeID': [97, 75, 115, 116],
#     'Level1_ID': [97, 97, 97, 97],
#     'Level2_ID': [None, 75, 75, 75],
#     'Level3_ID': [None, None, 115, 115],
#     'Level4_ID': [97, 75, 115, 116],
#     'HierarchyDepth': [1, 2, 3, 4] # Depths corresponding to the granular IDs
#     # ... other DimType columns like names/labels
# }
# df_dim_type = pd.DataFrame(dim_data)
df_dim_type = pd.DataFrame(df_dim)

# --- Load Source Fact Data ---
# fact_data = """
# "id","x","y","mp","date","state","relevance","types"
# 26994486,15.059215,50.770240,"true","2025-04-01T17:04:00.0000+02:00",1,4,75
# 26994486,15.059215,50.770240,"true","2025-04-01T17:04:00.0000+02:00",1,4,97
# 26994486,15.059215,50.770240,"true","2025-04-01T17:04:00.0000+02:00",1,4,115
# 26994486,15.059215,50.770240,"true","2025-04-01T17:04:00.0000+02:00",1,4,116
# 26994487,15.1,50.8,"false","2025-04-01T18:00:00.0000+02:00",1,3,97
# 26994488,15.2,50.9,"true","2025-04-01T19:00:00.0000+02:00",2,5,75
# 26994488,15.2,50.9,"true","2025-04-01T19:00:00.0000+02:00",2,5,97
# """
# df_source_facts = pd.read_csv(io.StringIO(fact_data))
df_source_facts = pd.read_csv("C://Users//jirip//Documents//Developer//python//kriminalita//202504.csv", sep=",", encoding="utf-8")

# --- Data Type Conversions (Important!) ---
df_source_facts['date'] = pd.to_datetime(df_source_facts['date'])
df_source_facts['mp'] = df_source_facts['mp'].map({'true': True, 'false': False})
# Ensure 'types' column is numeric if it's not already
df_source_facts['types'] = pd.to_numeric(df_source_facts['types'])


# --- ETL Process to get one row per event with granular TypeSK ---

# 1. Merge source facts with the relevant DimType info
#    We need TypeSK and HierarchyDepth based on the 'types' column
df_merged = pd.merge(
    df_source_facts,
    df_dim_type[['SourceTypeID', 'HierarchyDepth', 'TypeSK']],
    left_on='types',
    right_on='SourceTypeID',
    how='inner'  # Assumes every 'types' value in facts exists in DimType
)

# 2. Identify the most granular row for each 'id'
#    Sort by event id and then descending by depth, then keep the first row per id.
df_merged_sorted = df_merged.sort_values(by=['id', 'HierarchyDepth'], ascending=[True, False])
df_granular_facts = df_merged_sorted.drop_duplicates(subset=['id'], keep='first')

# 3. Select columns for the final Fact Table structure
#    We need the original fact columns (excluding 'types' and 'SourceTypeID')
#    and the 'TypeSK' we got from the merge corresponding to the most granular type.
final_fact_columns = [
    'id', 'x', 'y', 'mp', 'date', 'state', 'relevance', # Original fact measures/degenerates
    'TypeSK'  # Foreign key to DimType
]
df_final_fact_table = df_granular_facts[final_fact_columns]

# --- Output ---
print("--- Source Facts ---")
print(df_source_facts.to_string())
print("\n--- Dimension Type (Relevant Columns) ---")
print(df_dim_type[['TypeSK', 'SourceTypeID', 'HierarchyDepth']].to_string())
print("\n--- Merged Data (Facts + Dim Info) ---")
print(df_merged.to_string())
print("\n--- Identified Granular Rows (Staging) ---")
print(df_granular_facts.to_string())
print("\n--- Final Fact Table Data (One row per event) ---")
print(df_final_fact_table.to_string())

In [ ]:
import pandas as pd


# Load your CSVs
types_df = pd.read_csv("C://Users//jirip//Documents//Developer//python//kriminalita//types.csv", sep=",", encoding="utf-8")
fact_df = pd.read_csv("C://Users//jirip//Documents//Developer//python//kriminalita//202504.csv", sep=",", encoding="utf-8")

# Create a mapping of each type_id to its ancestors
def get_ancestors(row):
    return set([pid for pid in [row['parent_id1'], row['parent_id2'], row['parent_id3']] if pid > 0])

type_ancestors = types_df.set_index('id').apply(get_ancestors, axis=1).to_dict()

# Group fact table by `id` (your event) and process its type_ids
def filter_most_granular(group):
    type_ids = set(group['types'])
    to_remove = set()

    for t1 in type_ids:
        for t2 in type_ids:
            if t1 == t2:
                continue
            if t1 in type_ancestors.get(t2, set()):
                to_remove.add(t1)

    # Keep only the most granular types
    filtered = type_ids - to_remove
    return pd.DataFrame({'id': [group['id'].iloc[0]] * len(filtered), 'types': list(filtered)})

# Apply filtering
filtered_fact_df = fact_df.groupby('id', group_keys=False).apply(filter_most_granular).reset_index(drop=True)

print(filtered_fact_df)


In [1]:
import pandas as pd

# Load your CSVs
types_df = pd.read_csv("C://Users//jirip//Documents//Developer//python//kriminalita//types.csv", sep=",", encoding="utf-8")

# Prepare lookup for type names/labels
type_lookup = types_df.set_index('id')[['name', 'label']].to_dict('index')

# Create expanded hierarchy table
def expand_hierarchy(row):
    ids = [row['parent_id1'], row['parent_id2'], row['parent_id3'], row['id']]
    labels = [type_lookup.get(i, {'label': None})['label'] if i > 0 else None for i in ids]
    names = [type_lookup.get(i, {'name': None})['name'] if i > 0 else None for i in ids]
    
    result = {}
    for level, (tid, label, name) in enumerate(zip(ids, labels, names), 1):
        result[f'level_{level}_id'] = tid if tid > 0 else None
        result[f'level_{level}_label'] = label
        result[f'level_{level}_name'] = name
    return pd.Series(result)

dimension_df = types_df.apply(expand_hierarchy, axis=1)
dimension_df = pd.concat([types_df[['id']], dimension_df, types_df[['name', 'label']]], axis=1)

# Save or inspect
dimension_df.to_csv('dim_type.csv', index=False)


In [2]:
# Fill missing level_3 data from level_4
mask = dimension_df['level_3_id'].isna()
dimension_df.loc[mask, ['level_3_id', 'level_3_label', 'level_3_name']] = dimension_df.loc[mask, ['level_4_id', 'level_4_label', 'level_4_name']].values

# Fill missing level_2 data from level_3
mask = dimension_df['level_2_id'].isna()
dimension_df.loc[mask, ['level_2_id', 'level_2_label', 'level_2_name']] = dimension_df.loc[mask, ['level_3_id', 'level_3_label', 'level_3_name']].values

# Fill missing level_1 data from level_2
mask = dimension_df['level_1_id'].isna()
dimension_df.loc[mask, ['level_1_id', 'level_1_label', 'level_1_name']] = dimension_df.loc[mask, ['level_2_id', 'level_2_label', 'level_2_name']].values
dimension_df.to_csv('dim_type.csv', index=False)




In [1]:
import pandas as pd
fact_df = pd.read_csv("C://Users//jirip//Documents//Developer//python//kriminalita//202504.csv", sep=",", encoding="utf-8")
fact_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310501 entries, 0 to 310500
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         310501 non-null  int64  
 1   x          310501 non-null  float64
 2   y          310501 non-null  float64
 3   mp         310501 non-null  bool   
 4   date       310501 non-null  object 
 5   state      310501 non-null  int64  
 6   relevance  310501 non-null  int64  
 7   types      310501 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(1)
memory usage: 16.9+ MB


In [ ]:
import pandas as pd

# Load your CSV
types_df = pd.read_csv("C://Users//jirip//Documents//Developer//python//kriminalita//types.csv", sep=",", encoding="utf-8")

# Prepare lookup for type names/labels
type_lookup = types_df.set_index('id')[['name', 'label']].to_dict('index')

# Expand hierarchy
def expand_hierarchy(row):
    ids = [row['parent_id1'], row['parent_id2'], row['parent_id3'], row['id']]
    labels = [type_lookup.get(i, {'label': None})['label'] if i > 0 else None for i in ids]
    names = [type_lookup.get(i, {'name': None})['name'] if i > 0 else None for i in ids]

    # Shift hierarchy upwards if missing
    combined = [
        (tid if tid > 0 else None, label, name)
        for tid, label, name in zip(ids, labels, names)
    ]
    # Remove empty levels
    compacted = [(tid, label, name) for tid, label, name in combined if tid is not None]

    # Fill up to 4 levels
    while len(compacted) < 4:
        compacted.append((None, None, None))

    # Build the result
    result = {}
    for level, (tid, label, name) in enumerate(compacted, 1):
        result[f'level_{level}_id'] = tid
        result[f'level_{level}_label'] = label
        result[f'level_{level}_name'] = name

    return pd.Series(result)

# Apply hierarchy expansion
dimension_df = types_df.apply(expand_hierarchy, axis=1)

# Combine with the original type id, name, and label
dimension_df = pd.concat([types_df[['id']], dimension_df, types_df[['name', 'label']]], axis=1)

# Save the result
dimension_df.to_csv('dim_type.csv', index=False)


In [ ]:
# Build full ancestor map recursively
from collections import defaultdict

parent_map = defaultdict(set)

def collect_ancestors(type_id):
    if type_id == 0 or pd.isna(type_id):
        return set()
    row = types_df[types_df['id'] == type_id]
    if row.empty:
        return set()
    
    direct_parents = set(row[['parent_id1', 'parent_id2', 'parent_id3']].values.flatten())
    direct_parents = {pid for pid in direct_parents if pid > 0}
    
    ancestors = set(direct_parents)
    for parent in direct_parents:
        ancestors |= collect_ancestors(parent)
    return ancestors

# Build a cache
type_ancestors = {tid: collect_ancestors(tid) for tid in types_df['id']}

# Your fact table
fact_df = pd.read_csv("C://Users//jirip//Documents//Developer//python//kriminalita//202504.csv", sep=",", encoding="utf-8")

# Deduplicate for most granular types
def filter_most_granular(group):
    type_ids = set(group['types'])
    to_remove = set()

    for t1 in type_ids:
        for t2 in type_ids:
            if t1 == t2:
                continue
            if t1 in type_ancestors.get(t2, set()):
                to_remove.add(t1)

    filtered = type_ids - to_remove
    return pd.DataFrame({'id': [group['id'].iloc[0]] * len(filtered), 'types': list(filtered), 'Longitude': group['x'].iloc[0], 'Latitude': group['y'].iloc[0], 'mp': group['mp'].iloc[0], 'date': group['date'].iloc[0], 'state': group['state'].iloc[0], 'relevance': group['relevance'].iloc[0]})

filtered_fact_df = fact_df.groupby('id', group_keys=False).apply(filter_most_granular).reset_index(drop=True)

# Save your cleaned fact table
filtered_fact_df.to_csv('fact_clean.csv', index=False)


In [ ]:
# Assign 'is_one' based on the order of 'types' within each 'id'
filtered_fact_df['is_one'] = filtered_fact_df.sort_values(by=['id', 'types']).groupby('id').cumcount() + 1

# Sort by 'id' and 'types' to ensure proper ordering
filtered_fact_df = filtered_fact_df.sort_values(by=['id', 'types']).reset_index(drop=True)

# Preview the updated DataFrame
print(filtered_fact_df.head())

# Save your cleaned fact table
filtered_fact_df.to_csv('fact_clean.csv', index=False)

In [ ]:
import pandas as pd
import re

# Define your file path
file_path = "C://Users//jirip//Documents//Developer//python//kriminalita//2023//2023_12_Prosinec_sest_01.xlsx" 
match = re.search(r"(\d{4})_(\d{2})", file_path)
year, month = match.groups() if match else (None, None)

# Define your column headers manually
columns = [
    'TSK', 'TSK_desc', 'Registrovano_pocet', 'Objasneno_pocet', 'Objasneno_procent',
    'Spachano_nezletilymi', 'Spachano_mladistvymi', 'Spachano_detmi', 'Spachano_opakovane', 'Spachano_cizinci',
    'Spachano_pod_vlivem', 'Spachano_pod_vlivem_alkoholu', 'Spachano_firmou', 'Objasneno_dodatecne', 'Objasneno_celkem',
    'Spachano_nezletilymi_celkem', 'Spachano_mladistvymi_celkem', 'Spachano_detmi_celkem', 'Spachano_opakovane_celkem', 'Spachano_cizinci_celkem',
    'Spachano_pod_vlivem_celkem', 'Spachano_pod_vlivem_alkoholu_celkem', 'Spachano_firmou_celkem',
     'Skoda_celkem_tis', 'Skoda_zajistena_tis'
]  # Should match number of columns (Y-A + 1 = 25)

# Load Excel file and get all sheet names
xlsx = pd.ExcelFile(file_path, engine='openpyxl')

# Container for all data
all_data = []

# Loop through each sheet
for sheet_name in xlsx.sheet_names:
    # Read the specific range (A7:Y304 -> rows 6 to 303 with zero indexing)
    df = pd.read_excel(xlsx, sheet_name=sheet_name, usecols='A:Y', skiprows=6, nrows=275, header=None)
    df.columns = columns
    df['Region'] = sheet_name
    df['Year'] = year
    df['Month'] = month
    all_data.append(df)
# Concatenate into one DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Preview
print(final_df.head())


In [ ]:
# PROCESS YEAR 2024

import pandas as pd
import os
import re
from datetime import datetime

# --- CONFIGURATION ---
folder_path = 'C://Users//jirip//Documents//Developer//python//kriminalita//2024'  # Replace with your folder path
columns = [
    'TSK', 'TSK_desc', 'Registrovano_pocet', 'Objasneno_pocet', 'Objasneno_procent',
    'Spachano_nezletilymi', 'Spachano_mladistvymi', 'Spachano_detmi', 'Spachano_opakovane', 'Spachano_cizinci',
    'Spachano_pod_vlivem', 'Spachano_pod_vlivem_alkoholu', 'Spachano_firmou', 'Objasneno_dodatecne', 'Objasneno_celkem',
    'Spachano_nezletilymi_celkem', 'Spachano_mladistvymi_celkem', 'Spachano_detmi_celkem', 'Spachano_opakovane_celkem', 'Spachano_cizinci_celkem',
    'Spachano_pod_vlivem_celkem', 'Spachano_pod_vlivem_alkoholu_celkem', 'Spachano_firmou_celkem',
     'Skoda_celkem_tis', 'Skoda_zajistena_tis'
]

# --- MAIN EXTRACTION ---
all_data = []

for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)

        # Extract year and month from filename
        match = re.search(r"(\d{4})_(\d{2})", filename)
        if not match:
            continue  # Skip files that don't match the pattern
        year, month = map(int, match.groups())
        period = datetime(year, month, 1)

        # Load Excel file
        xlsx = pd.ExcelFile(file_path, engine='openpyxl')

        for sheet_name in xlsx.sheet_names:
            df = pd.read_excel(
                xlsx,
                sheet_name=sheet_name,
                usecols="A:Y",
                skiprows=6,
                nrows=298,
                header=None
            )
            df.columns = columns
            df['Region'] = sheet_name
            df['Period'] = period
            all_data.append(df)

# Combine all data into one DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Preview
print(final_df.head())

# Optional: Save output
final_df.to_csv("combined_data_2024.csv", index=False)
# final_df.to_excel("combined_data.xlsx", index=False)


In [ ]:
# PROCESS YEAR 2024

import pandas as pd
import os
import re
from datetime import datetime

# --- CONFIGURATION ---
folder_path = 'C://Users//jirip//Documents//Developer//python//kriminalita//2024'  # Replace with your folder path
columns = [
    'TSK', 'TSK_desc', 'Registrovano_pocet', 'Objasneno_pocet', 'Objasneno_procent',
    'Spachano_nezletilymi', 'Spachano_mladistvymi', 'Spachano_detmi', 'Spachano_opakovane', 'Spachano_cizinci',
    'Spachano_pod_vlivem', 'Spachano_pod_vlivem_alkoholu', 'Spachano_firmou', 'Objasneno_dodatecne', 'Objasneno_celkem',
    'Spachano_nezletilymi_celkem', 'Spachano_mladistvymi_celkem', 'Spachano_detmi_celkem', 'Spachano_opakovane_celkem', 'Spachano_cizinci_celkem',
    'Spachano_pod_vlivem_celkem', 'Spachano_pod_vlivem_alkoholu_celkem', 'Spachano_firmou_celkem',
     'Skoda_celkem_tis', 'Skoda_zajistena_tis'
]

# --- MAIN EXTRACTION ---
all_data = []

for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)

        # Extract year and month from filename
        match = re.search(r"(\d{4})_(\d{2})", filename)
        if not match:
            continue  # Skip files that don't match the pattern
        year, month = map(int, match.groups())
        period = datetime(year, month, 1)

        # Load Excel file
        xlsx = pd.ExcelFile(file_path, engine='openpyxl')

        for sheet_name in xlsx.sheet_names:
            df = pd.read_excel(
                xlsx,
                sheet_name=sheet_name,
                usecols="A:Y",
                skiprows=306,
                nrows=12,
                header=None
            )
            df.columns = columns
            df['Region'] = sheet_name
            df['Period'] = period
            all_data.append(df)

# Combine all data into one DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Preview
print(final_df.head())

# Optional: Save output
final_df.to_csv("AGG_data_2024.csv", index=False)
# final_df.to_excel("combined_data.xlsx", index=False)


In [ ]:
# PROCESS YEAR 2023

import pandas as pd
import os
import re
from datetime import datetime

# --- CONFIGURATION ---
folder_path = 'C://Users//jirip//Documents//Developer//python//kriminalita//2023'  # Replace with your folder path
columns = [
    'TSK', 'TSK_desc', 'Registrovano_pocet', 'Objasneno_pocet', 'Objasneno_procent',
    'Spachano_nezletilymi', 'Spachano_mladistvymi', 'Spachano_detmi', 'Spachano_opakovane', 'Spachano_cizinci',
    'Spachano_pod_vlivem', 'Spachano_pod_vlivem_alkoholu', 'Spachano_firmou', 'Objasneno_dodatecne', 'Objasneno_celkem',
    'Spachano_nezletilymi_celkem', 'Spachano_mladistvymi_celkem', 'Spachano_detmi_celkem', 'Spachano_opakovane_celkem', 'Spachano_cizinci_celkem',
    'Spachano_pod_vlivem_celkem', 'Spachano_pod_vlivem_alkoholu_celkem', 'Spachano_firmou_celkem',
     'Skoda_celkem_tis', 'Skoda_zajistena_tis'
]

# --- MAIN EXTRACTION ---
all_data = []

for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)

        # Extract year and month from filename
        match = re.search(r"(\d{4})_(\d{2})", filename)
        if not match:
            continue  # Skip files that don't match the pattern
        year, month = map(int, match.groups())
        period = datetime(year, month, 1)

        # Load Excel file
        xlsx = pd.ExcelFile(file_path, engine='openpyxl')

        for sheet_name in xlsx.sheet_names:
            df = pd.read_excel(
                xlsx,
                sheet_name=sheet_name,
                usecols="A:Y",
                skiprows=6,
                nrows=275,
                header=None
            )
            df.columns = columns
            df['Region'] = sheet_name
            df['Period'] = period
            all_data.append(df)

# Combine all data into one DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Preview
print(final_df.head())

# Optional: Save output
final_df.to_csv("combined_data_2023.csv", index=False)
# final_df.to_excel("combined_data.xlsx", index=False)


In [ ]:
# PROCESS YEAR 2023

import pandas as pd
import os
import re
from datetime import datetime

# --- CONFIGURATION ---
folder_path = 'C://Users//jirip//Documents//Developer//python//kriminalita//2023'  # Replace with your folder path
columns = [
    'TSK', 'TSK_desc', 'Registrovano_pocet', 'Objasneno_pocet', 'Objasneno_procent',
    'Spachano_nezletilymi', 'Spachano_mladistvymi', 'Spachano_detmi', 'Spachano_opakovane', 'Spachano_cizinci',
    'Spachano_pod_vlivem', 'Spachano_pod_vlivem_alkoholu', 'Spachano_firmou', 'Objasneno_dodatecne', 'Objasneno_celkem',
    'Spachano_nezletilymi_celkem', 'Spachano_mladistvymi_celkem', 'Spachano_detmi_celkem', 'Spachano_opakovane_celkem', 'Spachano_cizinci_celkem',
    'Spachano_pod_vlivem_celkem', 'Spachano_pod_vlivem_alkoholu_celkem', 'Spachano_firmou_celkem',
     'Skoda_celkem_tis', 'Skoda_zajistena_tis'
]

# --- MAIN EXTRACTION ---
all_data = []

for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)

        # Extract year and month from filename
        match = re.search(r"(\d{4})_(\d{2})", filename)
        if not match:
            continue  # Skip files that don't match the pattern
        year, month = map(int, match.groups())
        period = datetime(year, month, 1)

        # Load Excel file
        xlsx = pd.ExcelFile(file_path, engine='openpyxl')

        for sheet_name in xlsx.sheet_names:
            df = pd.read_excel(
                xlsx,
                sheet_name=sheet_name,
                usecols="A:Y",
                skiprows=283,
                nrows=12,
                header=None
            )
            df.columns = columns
            df['Region'] = sheet_name
            df['Period'] = period
            all_data.append(df)

# Combine all data into one DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Preview
print(final_df.head())

# Optional: Save output
final_df.to_csv("AGG_data_2023.csv", index=False)
# final_df.to_excel("combined_data.xlsx", index=False)


In [ ]:
# DE-YTD
import pandas as pd

# Load your CSV
df = pd.read_csv("C://Users//jirip//Documents//Developer//python//kriminalita//combined_data_2024.csv", parse_dates=["Period"])

# Define identifier columns and columns to transform
id_cols = ['Region', 'TSK', 'TSK_desc', 'Period']
value_cols = [col for col in df.columns if col not in id_cols]

# Sort for diffing
df = df.sort_values(by=['Region', 'TSK', 'Period'])

# Apply .diff() to get monthly deltas
df[value_cols] = df.groupby(['Region', 'TSK'])[value_cols].diff().fillna(df[value_cols])

# Optional: round or convert types if needed
df[value_cols] = df[value_cols].round(2)

# Preview
print(df.head())


In [ ]:
# DE-YTD
import pandas as pd
df= pd.read_csv("C://Users//jirip//Documents//Developer//python//kriminalita//AGG_data_2024.csv", parse_dates=["Period"])

# List of YTD columns to transform (excluding identifier and dimension columns)
ytd_columns = [
    col for col in df.columns
    if col not in ['Region', 'TSK', 'TSK_desc', 'Period']
]

# Sort for correct chronological operations
df = df.sort_values(by=['Region', 'TSK', 'Period'])

# Group and transform YTD to monthly deltas
def convert_ytd_to_monthly(group):
    monthly = group[ytd_columns].diff().fillna(group[ytd_columns])
    monthly.columns = [f"{col}_monthly" for col in monthly.columns]
    return pd.concat([group[['Region', 'TSK', 'TSK_desc', 'Period']], monthly], axis=1)

# Apply transformation
df_monthly = df.groupby(['Region', 'TSK'], group_keys=False).apply(convert_ytd_to_monthly)

# Preview
print(df_monthly.head())
